In [13]:
import torch
import mano
from mano.utils import Mesh

model_path = './_DATA/data/mano'
n_comps = 45
batch_size = 10

rh_model = mano.load(model_path=model_path,
                     is_rhand= True,
                     num_pca_comps=n_comps,
                     batch_size=batch_size,
                     flat_hand_mean=False)

betas = torch.rand(batch_size, 10)*.1
pose = torch.rand(batch_size, n_comps)*.1
global_orient = torch.rand(batch_size, 3)
transl        = torch.rand(batch_size, 3)

output = rh_model(betas=betas,
                  global_orient=global_orient,
                  hand_pose=pose,
                  transl=transl,
                  return_verts=True,
                  return_tips = True)


h_meshes = rh_model.hand_meshes(output)
j_meshes = rh_model.joint_meshes(output)

In [23]:
import numpy as np
import plotly.graph_objects as go

def display_mesh_plotly(mesh):
    """
    mano.utils.Mesh 객체의 vertices와 faces를 Plotly의 Mesh3d로 변환하여,
    Notebook 내에서 인터랙티브하게 시각화합니다.
    """
    # mesh가 vertices와 faces 속성을 가지고 있다고 가정합니다.
    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)
    
    x, y, z = vertices[:, 0], vertices[:, 1], vertices[:, 2]
    # faces 배열의 각 행은 삼각형의 세 꼭짓점 인덱스를 나타냅니다.
    i = faces[:, 0]
    j = faces[:, 1]
    k = faces[:, 2]
    
    fig = go.Figure(data=[
        go.Mesh3d(
            x=x, y=y, z=z,
            i=i, j=j, k=k,
            opacity=0.50,
            color='lightblue',
            flatshading=True
        )
    ])
    fig.update_layout(
        title="3D Mesh Visualization with Perspective",
        scene=dict(
            aspectmode='data',
            camera=dict(
                projection=dict(type='perspective')  # 카메라를 perspective 투영으로 설정.
            )
        )
    )
    fig.show()

# 사용 예시:
display_mesh_plotly(h_meshes[0])

In [22]:
def display_joints_with_colors(joint_mesh):
    """
    MANO 모델에서 생성한 joint_mesh (예: j_meshes[0])의 관절 위치 정보를,
    21개의 서로 다른 색상으로 시각화합니다.
    
    인자:
        joint_mesh: 관절 Mesh 객체. .vertices 속성에 shape (21, 3)의 관절 좌표가 있어야 합니다.
    """
    import numpy as np
    import plotly.graph_objects as go

    # 관절 좌표 추출 (예상 shape: (21, 3))
    joints = np.array(joint_mesh.vertices)
    joints = output.joints[0]
    
    # 21개 관절에 사용할 21개의 색상 (원하는 색상으로 수정 가능)
    colors = [
        'red', 'green', 'blue', 'orange', 'purple',
        'cyan', 'magenta', 'yellow', 'brown', 'pink',
        'gray', 'olive', 'lime', 'maroon', 'navy',
        'teal', 'silver', 'gold', 'black', 'indigo', 'coral'
    ]
    
    fig = go.Figure()
    
    # 각 관절마다 다른 색상의 마커를 추가합니다.
    for i in range(joints.shape[0]):
        fig.add_trace(go.Scatter3d(
            x=[joints[i, 0]],
            y=[joints[i, 1]],
            z=[joints[i, 2]],
            mode='markers',
            marker=dict(size=8, color=colors[i]),
            name=f'Joint {i}'
        ))
    
    fig.update_layout(
        title="Colored Hand Skeleton Joints",
        scene=dict(aspectmode='data')
    )
    fig.show()

# 사용 예시:
# j_meshes는 MANO 모델 출력으로부터 생성된 관절 Mesh 리스트라고 가정할 때,
# 첫 번째 샘플의 관절 Mesh에 대해 색상을 표현해봅니다.
display_joints_with_colors(j_meshes[0])

In [41]:
def display_hand_mesh_with_skeleton(hand_mesh, core_joints):
    """
    손 메쉬와 함께 핵심 손 스켈레톤 (21개 key joint) 점들을 overlay 시각화합니다.
    
    인자:
        hand_mesh: mano hand mesh 객체. vertices와 faces 속성이 있어야 합니다.
        core_joints: (21, 3) shape의 핵심 관절 좌표.
                     보통 output.joints[0] 와 같이 사용됩니다.
    """
    import numpy as np
    import plotly.graph_objects as go
    
    # 손 메쉬 데이터 추출
    vertices = np.array(hand_mesh.vertices)
    faces = np.array(hand_mesh.faces)
    x, y, z = vertices[:, 0], vertices[:, 1], vertices[:, 2]
    i = faces[:, 0]
    j_vals = faces[:, 1]
    k = faces[:, 2]
    
    # Plotly figure 생성 및 손 메쉬 추가 (투시 렌더링)
    fig = go.Figure()
    fig.add_trace(go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j_vals, k=k,
        opacity=0.3,          # 내부가 보이도록 불투명도 낮춤
        color='lightblue',
        flatshading=True,
        name="Hand Mesh"
    ))
    
    # 핵심 관절 좌표 (21개) 오버레이
    core_joints = np.array(core_joints)
    # 21개 관절에 사용할 색상 리스트 (원하는 색상으로 수정 가능)
    colors = [
        'red', 'green', 'blue', 'orange', 'purple',
        'cyan', 'magenta', 'yellow', 'brown', 'pink',
        'gray', 'olive', 'lime', 'maroon', 'navy',
        'teal', 'silver', 'gold', 'black', 'indigo', 'coral'
    ]
    
    for idx in range(core_joints.shape[0]):
        fig.add_trace(go.Scatter3d(
            x=[core_joints[idx, 0]],
            y=[core_joints[idx, 1]],
            z=[core_joints[idx, 2]],
            mode='markers',
            marker=dict(size=6, color=colors[idx % len(colors)]),
            name=f'Joint {idx}'
        ))
    
    # 카메라 투시 설정
    fig.update_layout(
        title="Hand Mesh with Core Skeleton Points",
        scene=dict(
            aspectmode='data',
            camera=dict(
                projection=dict(type='perspective')  # 투시 (perspective) 방식
            )
        )
    )
    
    fig.show()


# 사용 예시:
# MANO 모델 출력의 첫번째 샘플에 대한 손 메쉬와, 핵심 관절 21개 점을 함께 시각화합니다.
display_hand_mesh_with_skeleton(h_meshes[0], output.joints[0])

In [40]:
def display_skeleton_with_local_axes(joints, hand_mesh, axis_length=0.05):
    """
    각 관절의 위치에서, 해당 관절에 고유한 로컬 좌표계(3축)를 시각화합니다.
    
    각 관절에 대해:
      - y축: 관절의 뼈(bone) 방향. 
             (자식이 존재하면, 자식 방향의 평균; tip인 경우에는 부모와의 차이 이용)
      - z축: 손바닥 안쪽 피부로 수직한 방향 (여기서는 global palm normal을 사용)
      - x축: 오른손 좌표계에 맞게 재계산 (x = normalize(cross(y, global_palm_normal)))
             이후 z = normalize(cross(x, y))로 정규 직교 좌표계를 구성.
    
    인자:
      joints: (21, 3) 형태의 numpy 배열 또는 리스트. 각 행은 관절의 (x, y, z) 좌표.
      axis_length: 각 축을 그릴 길이. (단위는 joints 좌표와 동일)
    """
    import numpy as np
    import plotly.graph_objects as go

    # numpy array로 변환
    joints = np.array(joints)
        # 손 메쉬 데이터 추출
    vertices = np.array(hand_mesh.vertices)
    faces = np.array(hand_mesh.faces)
    x, y, z = vertices[:, 0], vertices[:, 1], vertices[:, 2]
    i = faces[:, 0]
    j_vals = faces[:, 1]
    k = faces[:, 2]
    

    # 정규화 함수
    def normalize(v):
        norm = np.linalg.norm(v)
        return v / norm if norm > 1e-8 else v
    
    # MANO hand skeleton hierarchy 
    # 0 (palm) -
    # 13-14-15-16 (thumb)
    # 1-2-3-17 (index)
    # 4-5-6-18 (middle)
    # 10-11-12-19 (ring)
    # 7-8-9-20 (little)

    child_dict = {
        0: [13, 1, 4, 10, 7],
        1: [2],
        2: [3],
        3: [17],
        4: [5],
        5: [6],
        6: [18],
        7: [8],
        8: [9],
        9: [20],
        10: [11],
        11: [12],
        12: [19],
        13: [14],
        14: [15],
        15: [16]
    }
    parent_dict = {
        1: 0, 2: 1, 3: 2, 4: 0,
        5: 4, 6: 5, 7: 0, 8: 7,
        9: 8, 10: 0, 11: 10, 12: 11,
        13: 0, 14: 13, 15: 14, 16: 15,
        17: 3, 18: 6, 19: 12, 20: 9
    }
    
    # 전역 palm normal 계산 (손목, 인덱스 기저, 소지 기저 이용)
    v1 = joints[1] - joints[0]    # 예: index finger base 방향
    v2 = joints[7] - joints[0]   # 예: little finger base 방향
    global_palm_normal = normalize(np.cross(v1, v2))
    # 필요에 따라 palm normal의 방향을 반전할 수 있습니다.
    # global_palm_normal = -global_palm_normal
    
    fig = go.Figure()

    # Plotly figure 생성 및 손 메쉬 추가 (투시 렌더링)
    fig.add_trace(go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j_vals, k=k,
        opacity=0.3,          # 내부가 보이도록 불투명도 낮춤
        color='lightblue',
        flatshading=True,
        name="Hand Mesh"
    ))
    
    
    # 관절 위치(마커) 표시
    fig.add_trace(go.Scatter3d(
        x=joints[:, 0],
        y=joints[:, 1],
        z=joints[:, 2],
        mode='markers',
        marker=dict(size=5, color='black'),
        name='Joints'
    ))
    
    # 각 관절마다 로컬 좌표계(축) 계산 및 표시.
    for i in range(joints.shape[0]):
        joint_pos = joints[i]
        
        # 뼈 방향 (y축) 결정:
        if i in child_dict and len(child_dict[i]) > 0:
            # 자식이 여러 개면 평균 방향 사용 (예를 들어 손목에는 여러 자식이 있음)
            directions = [joints[c] - joint_pos for c in child_dict[i]]
            y_axis = normalize(np.mean(directions, axis=0))
        elif i in parent_dict:
            # tip인 경우, 부모와의 차이 사용
            y_axis = normalize(joint_pos - joints[parent_dict[i]])
        else:
            # 특별한 경우 (예: wrist인데 자식도 없다면) 기본값 사용
            y_axis = np.array([0, 1, 0])
        
        # 로컬 x축: y축과 global palm normal의 외적
        x_axis = normalize(np.cross(y_axis, global_palm_normal))
        # 재정규화된 z축: x축과 y축의 외적 (정확한 직교 좌표계 구성)
        z_axis = normalize(np.cross(x_axis, y_axis))
        
        # 각 축 선분 그리기
        # x축: magenta
        fig.add_trace(go.Scatter3d(
            x=[joint_pos[0], joint_pos[0] + axis_length * x_axis[0]],
            y=[joint_pos[1], joint_pos[1] + axis_length * x_axis[1]],
            z=[joint_pos[2], joint_pos[2] + axis_length * x_axis[2]],
            mode='lines',
            line=dict(color='magenta', width=4),
            showlegend=False
        ))
        # y축: green (뼈 방향)
        fig.add_trace(go.Scatter3d(
            x=[joint_pos[0], joint_pos[0] + axis_length * y_axis[0]],
            y=[joint_pos[1], joint_pos[1] + axis_length * y_axis[1]],
            z=[joint_pos[2], joint_pos[2] + axis_length * y_axis[2]],
            mode='lines',
            line=dict(color='green', width=4),
            showlegend=False
        ))
        # z축: blue (손바닥 안쪽 방향)
        fig.add_trace(go.Scatter3d(
            x=[joint_pos[0], joint_pos[0] + axis_length * z_axis[0]],
            y=[joint_pos[1], joint_pos[1] + axis_length * z_axis[1]],
            z=[joint_pos[2], joint_pos[2] + axis_length * z_axis[2]],
            mode='lines',
            line=dict(color='blue', width=4),
            showlegend=False
        ))
    
    fig.update_layout(
        title="Hand Skeleton with Local Axes per Joint",
        scene=dict(
            aspectmode='data',
            camera=dict(
                projection=dict(type='perspective')
            )
        )
    )
    
    fig.show()

# 사용 예시:
# output.joints[0]가 (21,3) 형태의 핵심 hand skeleton 좌표라고 가정합니다.
display_skeleton_with_local_axes(output.joints[0], h_meshes[0], axis_length=0.01)